In [157]:
import pandas as pd
import dask.dataframe as dd
import numpy as np
from pandas.api.types import is_numeric_dtype

In [9]:
from ..Utils.cols_list import *

ImportError: attempted relative import with no known parent package

In [6]:
cols_list = [
    'NU_ANO_CENSO',
    'SG_UF',
    'CO_MUNICIPIO',
    'NO_MUNICIPIO',
    'CO_ENTIDADE',
    'TP_DEPENDENCIA',
    'TP_LOCALIZACAO',
    'TP_SITUACAO_FUNCIONAMENTO',
    'IN_AGUA_POTAVEL',
    'IN_ENERGIA_INEXISTENTE',
    'IN_ESGOTO_INEXISTENTE',
    'IN_AREA_VERDE',
    'IN_BANHEIRO',
    'IN_BIBLIOTECA',
    'IN_LABORATORIO_INFORMATICA',
    'IN_INTERNET',
    'IN_INTERNET_APRENDIZAGEM',
    'IN_INTERNET_ADMINISTRATIVO',
    'IN_INTERNET_COMUNIDADE',
    'IN_ACESSO_INTERNET_COMPUTADOR',
    'IN_ACES_INTERNET_DISP_PESSOAIS',
    'TP_REDE_LOCAL',
    'IN_BANDA_LARGA',
    'IN_EXAME_SELECAO',
    'QT_DOC_MED'
]

In [200]:
censo = dd.read_csv(
   '../../Dados/Censo Escolar/2019/dados/microdados_ed_basica_2019.csv', 
    sep=';', 
    encoding='latin-1',
    usecols=cols_list,
    assume_missing=True,
      dtype={'CO_MUNICIPIO': 'object',
             'CO_ENTIDADE': 'object',
             'NU_ANO_CENSO': 'int64'}
    )

In [199]:
censo.head()

,NU_ANO_CENSO,SG_UF,NO_MUNICIPIO,CO_MUNICIPIO,CO_ENTIDADE,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_SITUACAO_FUNCIONAMENTO,IN_AGUA_POTAVEL,IN_ENERGIA_INEXISTENTE,...,IN_INTERNET,IN_INTERNET_ADMINISTRATIVO,IN_INTERNET_APRENDIZAGEM,IN_INTERNET_COMUNIDADE,IN_ACESSO_INTERNET_COMPUTADOR,IN_ACES_INTERNET_DISP_PESSOAIS,TP_REDE_LOCAL,IN_BANDA_LARGA,IN_EXAME_SELECAO,QT_DOC_MED
0,2019.0,RO,Alta Floresta D'Oeste,1100015,11022558.0,2.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,NaN,0.0,0.0
1,2019.0,RO,Alta Floresta D'Oeste,1100015,11024275.0,2.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,2019.0,RO,Alta Floresta D'Oeste,1100015,11024291.0,3.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,NaN,0.0,0.0
3,2019.0,RO,Alta Floresta D'Oeste,1100015,11024372.0,3.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7,2019.0,RO,Alta Floresta D'Oeste,1100015,11024666.0,3.0,2.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Inferindo medidas de conectividade / acesso à tecnologia nas escolas e agregando um score à nível de município para matching com outras bases de dados

Realizar agregações:
- Por município
- Por tipo de dependência
- Por tipo de localização
Medindo as variáveis avaliadas como explicativas e criando um score para cada nas agregações descritas.

In [201]:
censo = censo[censo['TP_SITUACAO_FUNCIONAMENTO']==1]

In [193]:
def get_agg(df, index_cols, agg_funs):
    '''
    Aggregates dataframe columns by a given list of agg functions.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe to be aggregated.
    index_cols : list
        List of columns to be aggregated.
    agg_funs : dict
        dict of aggregation functions to be applied in each key pair.

    Returns
    -------
    ddf : dask.DataFrame
        Dask dataframe with aggregated columns.
    '''
    
    df_sum=df.groupby(index_cols).agg(agg_funs['sum'])
    df_sum = df_sum.reset_index()
    df_count=df.groupby(index_cols).agg(agg_funs['count'])
    df_count = df_count.reset_index()

    # Removing zero values to avoid division by zero error
    for col in df_count:
        df_count[col] = df_count[col].apply(lambda x: np.nan if x == 0 else x)

    # Applying percentage function to each numeric column
    numeric_cols = [col for col in df_sum if col not in index_cols]
    for col in numeric_cols:
        df_sum[col] = df_sum[col] / df_count[col]

    del df_count

    df_median = pd.DataFrame(df, columns=cols_list).groupby(index_cols).agg(agg_funs['median'])
    df_median = df_median.reset_index()
    ddf_median = dd.from_pandas(df_median, npartitions=1)

    del df_median

    # Joining dataframes with all aggregated values
    final_df = dd.merge(df_sum, ddf_median, on=index_cols, how='outer')
    #df_median.columns = [col + '_MED' for col in df_median.columns]
    #df_sum.columns = ['_'.join(col) for col in df.columns]

    return final_df


In [145]:
agg_dict = {
    'sum':{
        'IN_AGUA_POTAVEL':'sum',
        'IN_ENERGIA_INEXISTENTE':'sum',
        'IN_ESGOTO_INEXISTENTE':'sum',
        'IN_AREA_VERDE':'sum',
        'IN_BANHEIRO':'sum',
        'IN_BIBLIOTECA':'sum',
        'IN_LABORATORIO_INFORMATICA':'sum',
        'IN_INTERNET':'sum',
        'IN_INTERNET_APRENDIZAGEM':'sum',
        'IN_INTERNET_ADMINISTRATIVO':'sum',
        'IN_INTERNET_COMUNIDADE':'sum',
        'IN_ACESSO_INTERNET_COMPUTADOR':'sum',
        'IN_ACES_INTERNET_DISP_PESSOAIS':'sum',
        'IN_BANDA_LARGA':'sum',
        'IN_EXAME_SELECAO':'sum',
    },
    'count':{
        'IN_AGUA_POTAVEL':'count',
        'IN_ENERGIA_INEXISTENTE':'count',
        'IN_ESGOTO_INEXISTENTE':'count',
        'IN_AREA_VERDE':'count',
        'IN_BANHEIRO':'count',
        'IN_BIBLIOTECA':'count',
        'IN_LABORATORIO_INFORMATICA':'count',
        'IN_INTERNET':'count',
        'IN_INTERNET_APRENDIZAGEM':'count',
        'IN_INTERNET_ADMINISTRATIVO':'count',
        'IN_INTERNET_COMUNIDADE':'count',
        'IN_ACESSO_INTERNET_COMPUTADOR':'count',
        'IN_ACES_INTERNET_DISP_PESSOAIS':'count',
        'IN_BANDA_LARGA':'count',
        'IN_EXAME_SELECAO':'count',
    },
    'median':{
        'QT_DOC_MED':'median',
        'TP_REDE_LOCAL':'median'
    }
}

In [202]:
index_cols = ['NU_ANO_CENSO', 'SG_UF', 'CO_MUNICIPIO', 'TP_DEPENDENCIA', 'TP_LOCALIZACAO', 'TP_SITUACAO_FUNCIONAMENTO']
teste_dask = get_agg(censo, index_cols, agg_funs=agg_dict)

/home/joao/anaconda3/envs/scraping/lib/python3.9/site-packages/dask/dataframe/core.py:3997: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('NU_ANO_CENSO', 'int64'))

  warnings.warn(meta_warning(meta))
/home/joao/anaconda3/envs/scraping/lib/python3.9/site-packages/dask/dataframe/core.py:3997: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(f

In [195]:
teste_dask.head()

,NU_ANO_CENSO,SG_UF,CO_MUNICIPIO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_SITUACAO_FUNCIONAMENTO,IN_AGUA_POTAVEL,IN_ENERGIA_INEXISTENTE,IN_ESGOTO_INEXISTENTE,IN_AREA_VERDE,...,IN_INTERNET,IN_INTERNET_APRENDIZAGEM,IN_INTERNET_ADMINISTRATIVO,IN_INTERNET_COMUNIDADE,IN_ACESSO_INTERNET_COMPUTADOR,IN_ACES_INTERNET_DISP_PESSOAIS,IN_BANDA_LARGA,IN_EXAME_SELECAO,QT_DOC_MED,TP_REDE_LOCAL
0,2021.0,AC,1200013,2.0,1.0,1.0,1.000000,0.0,0.0,0.500000,...,1.000000,0.500000,0.500000,0.0,0.5,0.0,1.000000,0.0,NaN,NaN
1,2021.0,AC,1200013,2.0,2.0,1.0,1.000000,0.0,0.0,0.333333,...,1.000000,0.000000,1.000000,0.0,0.0,0.0,0.333333,0.0,NaN,NaN
2,2021.0,AC,1200013,3.0,1.0,1.0,0.666667,0.0,0.0,0.666667,...,1.000000,0.666667,1.000000,0.0,0.0,0.0,0.666667,0.0,NaN,NaN
3,2021.0,AC,1200013,3.0,2.0,1.0,1.000000,0.0,0.0,0.000000,...,0.666667,0.000000,0.666667,0.0,0.0,0.0,0.000000,0.0,NaN,NaN
4,2021.0,AC,1200054,2.0,1.0,1.0,1.000000,0.0,0.0,1.000000,...,1.000000,0.000000,1.000000,0.0,1.0,1.0,1.000000,0.0,NaN,NaN
